# NOTE 1!

The day2greg notebook is developed by https://github.com/gmerritt123 following the notebook developed by Jonny Williams. This notebook is a modification of gmerritt123 notebook to convert 360 and 365 day calendar netcdf to gregorian calendar. This notebook uses xarray instead of netCDF4 package to explore the NetCDF file. It also creates a new NetCDF file after the modification into Gregorian calendar. 


# NOTE 2!
Please do well to visit their pages and assign due credit to **https://github.com/jonnyhtw/360day2greg/tree/main, https://eartharxiv.org/repository/view/5645/ and https://github.com/gmerritt123 ** . 

I also utilise **ChatGPT from OpenAI** for debugging errors and streamlining the codes to make it more user friendly.

The notebook can work with **both single or multiple** files, the files must be present in the appropriate folders. **Don't mix** 360_day calendar files(eg. HadGEM or UKESM1 models) in the **same folder** as 365_day calendar files.

Finally, the original code in this notebook from __gmerritt123__ are in Markdown hereafter to serve as a reference

## day2greg

This notebook implements the methodology dev'd by Jonny Williams to adjust 365 day and 360 day calendar climate datasets to standard gregorian. It improves on the original notebook by standardizing the routine, leveraging numpy's array functions for fast vectorized operations, and using far fewer dependencies.

See 
- https://github.com/jonnyhtw/360day2greg/tree/main ; and
- https://eartharxiv.org/repository/view/5645/

In [ ]:
# Code Commence from here

from netCDF4 import Dataset
import os
import sys
import numpy as np
import calendar
import xarray as xr

os.chdir(r'D:\Dufie_Enoch_CMIP6\ALL_MERGED_HAS_FUT\HadGEM3_GC31_LL_ssp245') #just my working dir

## Just two functions!

One to get an array of indices where to perform the interpolated insertions, and another to do them.

def getGregorianSeeds(num_yrs,y0,day_cal):
    ''' used to obtain array of random "seeds" for adding extra interpolated days within each year.
    args:
    num_yrs = total number of years
    y0 = year zero (e.g. 1999)
    day_cal = the number of days in the calendar you're using (e.g. 360 or 365)
    returns: 
        list of indices at which to perform "interpolated insertions"
    '''
    assert day_cal <= 365, 'getSeeds only configured for adding interpolated days, day_cal must be 365 or less'
    ed = (365 - day_cal) # how many extra days you need to add to a "normal" year
    ins = [] #running tally of indices to flag where interpolation should occur
    #starting in y0 for the spec'd number of years...
    for i,y in enumerate(range(y0,y0+num_yrs,1)):
        #check if leap year
        if calendar.isleap(y):
            b = ed+2 #b = number of divisions to apply to year
        else:
            b = ed+1
        sr = np.linspace(0,day_cal,b) #makes "bins" within the year by which to sample random seeds
        if len(sr) == 1: #special case just handling if you don't need to sample any extra days
            s = day_cal
        else:
            s = sr[1]-sr[0] #size of each bin 
        rs = np.round(np.random.rand(b-1)*s) #makes b-1 random numbers between 0 and 1 multiplied by the bin size to get an "offset" from sr
        seeds = (sr[:-1]+rs).astype(int) #adds the offsets to the divisions 
        ins.extend(seeds+day_cal*i) #extends the running tally by the seeds + the number of days elapsed up to the current year
    return ins

def getInterpolatedData(arr,ins):
    '''Inserts interpolated data into existing array given list of indices where to insert. DOES NOT ALTER EXISTING ARRAY
    If indice is in ins, an additional new value will be inserted set to "halfway" between the current value and the next value.  
    args:
        - arr: array of data to get extra days inserted to it, typically for transient grid will be of shape (time,col,row)
        - ins: list or array of of indices at which to do the interpolated insertion
    returns: new array of length len(arr) + len(ins) containing the interpolated values in the spec'd indices
        '''
    r = [] #new data
    for i,a in enumerate(arr):
        r.append(a)
        if i in ins:

            r.append((arr[i]+arr[i+1])/2)
    return np.array(r)

# example usage 1

- UKESM1 model is 360 day

#load net cdf of precip
fn = r'Precip\\pr_day_bccaqv2_anusplin300_ukesm1_0_ll_historical_ssp585_r1i1p1f2_gn_19500101_21001230_sub.nc'
d = Dataset(fn)
print('dataset calendar:')
print(d.variables['time'].calendar) #360 day.

va = d.variables['pr'][:].transpose((2,0,1)) #transposes variable from shape (col,row,time) into array (time,col,row)
ny = int(va.shape[0]/360) #number of years
y0 = 1950 #starting year

ins = getGregorianSeeds(num_yrs=ny,y0=1950,day_cal=360) #gets a list of indices in the entire dataset where to do the interpolated insertions
# print(ins)
out = getInterpolatedData(arr=va,ins=ins) #makes a new array with the interpolated arrays inserted

#check
print('number of original days:')
print(len(va))
print('number of inserted days:')
print(len(ins))
print('number of gregorian-based days from y0 for ny')
import pandas as pd #using this only for convenience of checking dates
print(len(pd.date_range(start='Jan 1, 1950',end='Dec 31 2100',freq='1D')))
print('number of days in updated dataset:')
print(out.shape[0])
del d 


In [67]:
print(d.variables['time'].calendar) #360 day.
calendar_attr = ds['time']


360_day


MODIFIED FROM HERE ONWARDS

# ALTERNATIVE 1  (To example 1 for 360_day)  : LOOPING FOR MULTIPLE FILES AND preserving SOURCE dimension attributes to output files

In [38]:
import os
import glob
import shutil  # Added import for shutil
import numpy as np
import calendar
import xarray as xr
import pandas as pd

#############################################################################################################
def getGregorianSeeds(num_yrs,y0,day_cal):
    ''' used to obtain array of random "seeds" for adding extra interpolated days within each year.
    args:
    num_yrs = total number of years
    y0 = year zero (e.g. 1999)
    day_cal = the number of days in the calendar you're using (e.g. 360 or 365)
    returns: 
        list of indices at which to perform "interpolated insertions"
    '''
    assert day_cal <= 365, 'getSeeds only configured for adding interpolated days, day_cal must be 365 or less'
    ed = (365 - day_cal) # how many extra days you need to add to a "normal" year
    ins = [] #running tally of indices to flag where interpolation should occur
    #starting in y0 for the spec'd number of years...
    for i,y in enumerate(range(y0,y0+num_yrs,1)):
        #check if leap year
        if calendar.isleap(y):
            b = ed+2 #b = number of divisions to apply to year
        else:
            b = ed+1
        sr = np.linspace(0,day_cal,b) #makes "bins" within the year by which to sample random seeds
        if len(sr) == 1: #special case just handling if you don't need to sample any extra days
            s = day_cal
        else:
            s = sr[1]-sr[0] #size of each bin 
        rs = np.round(np.random.rand(b-1)*s) #makes b-1 random numbers between 0 and 1 multiplied by the bin size to get an "offset" from sr
        seeds = (sr[:-1]+rs).astype(int) #adds the offsets to the divisions 
        ins.extend(seeds+day_cal*i) #extends the running tally by the seeds + the number of days elapsed up to the current year
    return ins

def getInterpolatedData(arr,ins):
    '''Inserts interpolated data into existing array given list of indices where to insert. DOES NOT ALTER EXISTING ARRAY
    If indice is in ins, an additional new value will be inserted set to "halfway" between the current value and the next value.  
    args:
        - arr: array of data to get extra days inserted to it, typically for transient grid will be of shape (time,col,row)
        - ins: list or array of of indices at which to do the interpolated insertion
    returns: new array of length len(arr) + len(ins) containing the interpolated values in the spec'd indices
        '''
    r = [] #new data
    for i,a in enumerate(arr):
        r.append(a)
        if i in ins:

            r.append((arr[i]+arr[i+1])/2)
    return np.array(r)
##################################################################################################################


def process_file(file_path, variable_name):
    ds = xr.open_dataset(file_path)

    #va = d.variables['pr'][:].transpose((2,0,1)) #transposes variable from shape (col,row,time) into array (time,col,row)
    va = ds[variable_name].values # I did not transpose because my data had the shape array (time,col,row)
    ny = int(va.shape[0] / 360) #number of years
    y0 = 1960  #starting year

    ins = getGregorianSeeds(num_yrs=ny, y0=1960, day_cal=360) #gets a list of indices in the entire dataset where to do the interpolated insertions
    # print(ins)
    out = getInterpolatedData(arr=va, ins=ins) #makes a new array with the interpolated arrays inserted
    

    # Define time coordinate and Copy Global attributes and variables definitions and attributes from source data
    time_coords = pd.date_range(start='Jan 1, 1960', end='Dec 31 2100', freq='1D')
    latitude_coords = ds['lat'].values
    longitude_coords = ds['lon'].values

    time_dim = xr.DataArray(time_coords, dims='time', name='time', attrs=ds['time'].attrs)
    latitude_dim = xr.DataArray(latitude_coords, dims='lat', name='lat', attrs=ds['lat'].attrs)
    longitude_dim = xr.DataArray(longitude_coords, dims='lon', name='lon', attrs=ds['lon'].attrs)

    ### Creation and Definition of the structure of the netcdf files 
    pr_da = xr.DataArray(out, dims=('time', 'lat', 'lon'), coords={'time': time_dim, 'lat': latitude_dim, 'lon': longitude_dim}, attrs=ds[variable_name].attrs)

    new_ds = xr.Dataset({variable_name: pr_da})

    # Copy global attributes from the original dataset
    for attr in ds.attrs:
        new_ds.attrs[attr] = ds.attrs[attr]

    # Save the new dataset to a NetCDF file
    new_filename = os.path.join(output_dir, f'output_{variable_name}_{os.path.basename(file_path)}') # You can Modify if desired 
    new_ds.to_netcdf(new_filename)

    # Check the new NetCDF file
    print(f'NetCDF file saved: {new_filename}')
    
    # Check You can comment out this part of the code
    print('number of original days:')
    print(len(va))
    print('number of inserted days:')
    print(len(ins))
    print('number of gregorian-based days from y0 for ny')
    
    print(len(pd.date_range(start='Jan 1, 1950',end='Dec 31 2100',freq='1D')))
    print('number of days in updated dataset:')
    #print(out.shape[0])
    
    # Close the original dataset
    ds.close()

# Specify the directory where your files are located
data_dir = '/media/peterrock/Volume/Dufie_Enoch_CMIP6/ALL_MERGED_HAS_FUT/HadGEM3_GC31_LL_ssp245' #MODIFY/CHANGE HERE

# Specify the output directory
output_dir = os.path.join(data_dir, 'output_360_corrected')        # OUTPUT FOLDER : You can Modify

# Use glob to select all files matching a specific pattern
file_pattern = os.path.join(data_dir, 'PRECIP', '*.nc')     #MODIFY/CHANGE HERE, The Folder name "PRECIP" can be changed to folder containing datasets with 360_day calendar
file_list = glob.glob(file_pattern)

# Check if the output directory exists, remove and recreate if needed
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

for fn in file_list:
    # Extract the variable name in the files first before running the process function
    variable_name = list(xr.open_dataset(fn).variables.keys())[3] # You can Modify [3], my Dataset is arranged as (time,lat,lon,pr)correspond to (0,1,2,3)
    process_file(fn, variable_name)


NetCDF file saved: /media/peterrock/Volume/Dufie_Enoch_CMIP6/ALL_MERGED_HAS_FUT/HadGEM3_GC31_LL_ssp245/output/output_pr_pr_day_HadGEM3-GC31-LL_ssp126_HIST_merged.nc
NetCDF file saved: /media/peterrock/Volume/Dufie_Enoch_CMIP6/ALL_MERGED_HAS_FUT/HadGEM3_GC31_LL_ssp245/output/output_tasmin_tasmin_day_HadGEM3-GC31-MM_ssp585_HIST_merged.nc


# example usage 2

- CMCC-ESM2 model is 365 day, want to insert an extra day of interpolated values randomly within each leap year

#load net cdf of precip
# fn = r'Precip\\pr_day_bccaqv2_anusplin300_cmcc_esm2_historical_ssp585_r1i1p1f1_gn_19500101_21001231_sub.nc' # For Windiws
d = Dataset(fn)
print('dataset calendar:')
print(d.variables['time'].calendar) #365 day.

va = d.variables['pr'][:].transpose((2,0,1)) #transposes variable from shape (col,row,time) into array (time,col,row)
ny = int(va.shape[0]/365) #number of years
y0 = 1950 #starting year

ins = getGregorianSeeds(num_yrs=ny,y0=1950,day_cal=365) #gets a list of indices in the entire dataset where to do the interpolated insertions
out = getInterpolatedData(arr=va,ins=ins) #makes a new array with the interpolated arrays inserted

#check
print('number of original days:')
print(len(va))
print('number of inserted days:')
print(len(ins))
print('number of gregorian-based days from y0 for ny')
import pandas as pd #using this only for convenience of checking dates
print(len(pd.date_range(start='Jan 1, 1950',end='Dec 31 2100',freq='1D')))
print('number of days in updated dataset:')
print(out.shape[0])

del d

# ALTERNATIVE 2 (To example 2) FOR 365 DAY CALENDAR

In [47]:
import os
import glob
import xarray as xr
import pandas as pd
import numpy as np
import calendar
import shutil

def getGregorianSeeds(num_yrs, y0, day_cal):
    assert day_cal <= 365, 'getSeeds only configured for adding interpolated days, day_cal must be 365 or less'
    ed = (365 - day_cal)
    ins = []

    for i, y in enumerate(range(y0, y0 + num_yrs, 1)):
        if calendar.isleap(y):
            b = ed + 2
        else:
            b = ed + 1
        sr = np.linspace(0, day_cal, b)
        if len(sr) == 1:
            s = day_cal
        else:
            s = sr[1] - sr[0]
        rs = np.round(np.random.rand(b - 1) * s)
        seeds = (sr[:-1] + rs).astype(int)
        ins.extend(seeds + day_cal * i)
    return ins

def getInterpolatedData(arr, ins):
    r = []
    for i, a in enumerate(arr):
        r.append(a)
        if i in ins:
            r.append((arr[i] + arr[i + 1]) / 2)
    return np.array(r)

os.chdir('/media/peterrock/Volume/Dufie_Enoch_CMIP6/ALL_MERGED_HAS_FUT/HadGEM3_GC31_LL_ssp245/') # Working Directory
data_dir = 'PRECIP_365'  # Update with the directory containing your files /Input FILES. #MODIFY/CHANGE HERE, The Folder name "PRECIP_365" can be changed to folder containing datasets with 365_day calendar

output_dir = 'output_365_corr'  # Update with the desired output directory. MODIFY if desired

# Check if the output directory exists and recreate it if necessary
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

file_pattern = os.path.join(data_dir, '*.nc')
file_list = glob.glob(file_pattern)

for fn in file_list:
    ds = xr.open_dataset(fn)

    # Dynamically fetch the variable name
    variable_name = list(ds.variables.keys())[3]

    va = ds[variable_name].transpose('time', 'lat', 'lon').values  # Use transpose to match the order of dimensions
    ny = int(va.shape[0] / 365)
    y0 = 1960  # Starting year of datasets    Change DATE

    ins = getGregorianSeeds(num_yrs=ny, y0=y0, day_cal=365)
    out = getInterpolatedData(arr=va, ins=ins)

    # Apply interpolation to xarray DataArray
    time_coords = pd.date_range(start=f'Jan 1, {y0}', end=f'Dec 31 2100', freq='1D')
    latitude_coords = ds['lat'].values
    longitude_coords = ds['lon'].values

    time_dim = xr.DataArray(time_coords, dims='time', name='time', attrs=ds['time'].attrs)
    latitude_dim = xr.DataArray(latitude_coords, dims='lat', name='lat', attrs=ds['lat'].attrs)
    longitude_dim = xr.DataArray(longitude_coords, dims='lon', name='lon', attrs=ds['lon'].attrs)

    pr_da = xr.DataArray(out, dims=('time', 'lat', 'lon'), coords={'time': time_dim, 'lat': latitude_dim, 'lon': longitude_dim}, attrs=ds[variable_name].attrs)

    new_ds = xr.Dataset({variable_name: pr_da})

    # Copy global attributes from the original dataset
    for attr in ds.attrs:
        new_ds.attrs[attr] = ds.attrs[attr]

    # Save the new dataset to a NetCDF file
    base_filename = os.path.basename(fn)
    new_filename = os.path.join(output_dir, f'output_{base_filename}')
    new_ds.to_netcdf(new_filename)

    # Check the new NetCDF file
    print(f'NetCDF file saved: {new_filename}')

    # Close the original dataset
    ds.close()


NetCDF file saved: output_365_corr/output_pr_day_CanESM5_ssp126_HIST_merged.nc
NetCDF file saved: output_365_corr/output_pr_day_CanESM5_ssp245_HIST_merged.nc
NetCDF file saved: output_365_corr/output_pr_day_CanESM5_ssp585_HIST_merged.nc
NetCDF file saved: output_365_corr/output_tas_day_CanESM5_ssp126_HIST_merged.nc
NetCDF file saved: output_365_corr/output_tas_day_CanESM5_ssp245_HIST_merged.nc
NetCDF file saved: output_365_corr/output_tas_day_CanESM5_ssp585_HIST_merged.nc
NetCDF file saved: output_365_corr/output_tasmax_day_CanESM5_ssp126_HIST_merged.nc
NetCDF file saved: output_365_corr/output_tasmax_day_CanESM5_ssp245_HIST_merged.nc
NetCDF file saved: output_365_corr/output_tasmax_day_CanESM5_ssp585_HIST_merged.nc
NetCDF file saved: output_365_corr/output_tasmin_day_CanESM5_ssp126_HIST_merged.nc
NetCDF file saved: output_365_corr/output_tasmin_day_CanESM5_ssp245_HIST_merged.nc
NetCDF file saved: output_365_corr/output_tasmin_day_CanESM5_ssp585_HIST_merged.nc
NetCDF file saved: output